In [1]:
!pip install -q -U google-generativeai

In [2]:
#Configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY="PEGAR A API KEY NO: https://aistudio.google.com/app/apikey E COLOCAR AQUI! (A API KEY NÃO DEVE SER COMPARTILHADA)"
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [23]:
#Exemplo de embedding
title = "Programação"
sample_text = ("Olá, mundo!")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.05644877, -0.0036741078, -0.049817905, 0.01462285, 0.024876552, 0.0013675805, 0.0127847595, 0.002690678, 0.0023724034, 0.012629108, -0.012215301, 0.008322696, 0.0026582568, -0.02146882, -0.013154641, -0.0041594356, 0.01009204, -0.0055135423, -0.030196555, -0.010153494, -2.7756583e-05, 0.0253723, -0.045175973, -0.043901406, 0.048206363, -0.00456389, 0.028124437, -0.060107473, 0.0123971915, 0.02172573, -0.019899307, -0.008257555, -0.031045934, 0.023243718, -0.0006687203, -0.053992186, -0.0059419516, 0.013155648, 0.0056957062, -0.014614413, 0.0064399997, -0.0674592, -0.034816545, 0.004219805, 0.030328754, -0.0050179227, -0.039407976, 0.02302259, 0.011739115, -0.09532268, 0.059122358, 0.045934048, 0.070867054, 0.013641087, -0.00023991003, -0.02437897, 0.09072209, -0.008046813, -0.06866771, 0.0062750075, 0.008262152, 0.03672604, 0.037519205, 0.066846795, -0.008175584, -0.01195272, -0.036065362, 0.03919363, 0.025223628, 0.010127029, -0.0007233331, -0.045079466, 0.018772297, 

In [42]:
DOCUMENT1 = {
    "title": "Operando o sistema de controle climático",
    "content": "Seu Googlecar possui um sistema de controle de temperatura que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central. Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuí-lo. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite selecionar o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Legal: O carro soprará ar frio para dentro do carro. Calor: O carro soprará ar quente para dentro do carro. Degelo: O carro soprará ar quente no para-brisa para descongelá-lo."}
DOCUMENT2 = {
    "title": "Tela sensível ao toque",
    "content": "Seu Googlecar possui uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle de temperatura. Para utilizar o display touchscreen, basta tocar no ícone desejado. Por exemplo, você pode tocar no ícone \"Navegação\" para obter rotas até seu destino ou tocar no ícone \"Música\" para reproduzir suas músicas favoritas."}
DOCUMENT3 = {
    "title": "Mudança de marcha",
    "content": "Seu Googlecar possui transmissão automática. Para mudar de marcha, basta mover a alavanca de câmbio para a posição desejada. Estacionar: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Reverso: Esta posição é usada para fazer backup. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive: Esta posição é usada para avançar. Baixo: Esta posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [43]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operando o sistema de controle climático,Seu Googlecar possui um sistema de controle de...
1,Tela sensível ao toque,Seu Googlecar possui uma grande tela sensível ...
2,Mudança de marcha,Seu Googlecar possui transmissão automática. P...


In [44]:
model = "models/embedding-001"

In [45]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [46]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operando o sistema de controle climático,Seu Googlecar possui um sistema de controle de...,"[-0.016749417, -0.033024393, -0.02863602, 0.04..."
1,Tela sensível ao toque,Seu Googlecar possui uma grande tela sensível ...,"[0.024339832, -0.040260006, 0.011001617, -0.00..."
2,Mudança de marcha,Seu Googlecar possui transmissão automática. P...,"[-0.009008808, -0.025796298, -0.01287553, 0.01..."


In [47]:
def gerar_e_buscar_consulta(consulta, base, model):
  embed_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embed_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [48]:
consulta = "Como faço para trocar a marcha em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar possui transmissão automática. Para mudar de marcha, basta mover a alavanca de câmbio para a posição desejada. Estacionar: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Reverso: Esta posição é usada para fazer backup. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive: Esta posição é usada para avançar. Baixo: Esta posição é usada para dirigir na neve ou em outras condições escorregadias.


In [52]:
config = {
    "candidate_count": 1,
    "temperature": 0,
}

In [53]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=config)
response = model_2.generate_content(prompt)
print(response.text)

**Guia de Marchas do Seu Googlecar:**

* **Estacionado:** Tá parado, né? Rodas travadas, não vai pra lugar nenhum.
* **Ré:** Pra dar aquela rézinha básica.
* **Neutro:** Tá parado, mas pode dar um gás se quiser.
* **Drive:** Bora pra frente!
* **Baixo:** Pra quando a coisa tá escorregadia, tipo neve ou gelo.
